In [2]:
import pandas as pd
import requests
import json
import csv
import os

In [3]:
# making a POST request to the API endpoint
# q contains a search query for a specific collection of items.
# extracted data and stored in a list called 'all_collections'

api_url = 'https://search2.raritysniper.com/multi_search?use_cache=true&x-typesense-api-key=KEY'
q ={
    "searches": [
        {
            "query_by": "name",
            "sort_by": "launchDate:desc",
            "highlight_full_fields": "name",
            "collection": "collections",
            "q": "*",
            "facet_by": "blockchain,oneDayVolume,supply,thirtyDayVolume,totalVolume,sevenDayVolume,floorPrice",
            "max_facet_values": 20,
            "page": 1,
            "per_page": 250
        }
    ]
}
headers =  {"Content-Type":"application/json"}
all_collections = []
for i in range(1, 10):
    q['searches'][0]['page']=i
    response = requests.post(api_url, data=json.dumps(q), headers=headers)
    r = response.json()
    collections = r['results'][0]['hits']
    for c in collections:
        all_collections.append(c['document'])
        # print(c['document'])

In [4]:
# create a DataFrame from the list of dictionaries

collection_df = pd.DataFrame(all_collections)
collection_df.to_csv('all_collections.csv', index=False)

In [5]:
# add 'https://raritysniper.com/' before collection_name in 'collectionSlug' column
# add 'assets_' before collection_name in 'collectionSlug' column

collection_df['collectionURL'] = 'https://raritysniper.com/' + collection_df['collectionSlug']
collection_df['assetsname'] = 'assets_' + collection_df['collectionSlug']
collection_df

,addedDate,allTimeAveragePrice,artStyle,blockchain,collectionSlug,contractAddress,discordUrl,floorPrice,fullyMinted,id,...,thirtyDayVolume,totalOnSale,totalSupply,totalVolume,twitterHandle,verified,websiteUrl,instagramHandle,collectionURL,assetsname
0,1677874451767,0.233838,uncategorized,ethereum,utopia-avatars,0x5f076e995290f3f9aea85fdd06d8fae118f2b75c,https://discord.gg/utopiaavatars,0.23770,True,2331,...,1360.937381,337.0,6628.0,1.360937e+03,UtopiaAvatars,True,https://utopia.io/,NaN,https://raritysniper.com/utopia-avatars,assets_utopia-avatars
1,1677839458430,0.182676,uncategorized,ethereum,spooky-boys-country-club,0xfd1076d80fff9dc702ae9fdfea0073467b9b3fb7,https://discord.gg/countryclub,0.02680,True,2300,...,14.623569,175.0,12999.0,1.890328e+03,SpookyBoysCC,True,https://www.spookyboys.io/,spookyboyscountryclub,https://raritysniper.com/spooky-boys-country-club,assets_spooky-boys-country-club
2,1677601223253,0.108530,uncategorized,ethereum,meka-drivers,0xdae97eafb92f988f268626e4b5120c3e6959bfc7,https://discord.gg/mekaverse,0.04680,True,2319,...,43.335376,478.0,8888.0,7.152146e+01,MekaVerse,True,https://themekaverse.com/,NaN,https://raritysniper.com/meka-drivers,assets_meka-drivers
3,1677360535916,0.053550,uncategorized,ethereum,hmn5,0x32fc5bcabc1f78308be11754493c49116c0fa35f,NaN,0.03490,True,2321,...,25.929888,261.0,6935.0,7.673676e+01,HMN5_NFT,True,http://hmn5.io/,NaN,https://raritysniper.com/hmn5,assets_hmn5
4,1677168044994,0.252243,uncategorized,ethereum,diesel-x-hape,0x4ce6c369516560b1e6fa49d41096746293c0a87f,NaN,0.13969,True,2320,...,136.715782,110.0,1024.0,1.367158e+02,HAPEsocial,True,http://diesel.hape.io,NaN,https://raritysniper.com/diesel-x-hape,assets_diesel-x-hape
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,1629003600000,3.426615,uncategorized,ethereum,cool-cats,0x1a92f7381b9f03921564a437210bb9396471050c,https://discord.gg/X6A4AXrKaR,2.10970,True,2,...,12918.796294,343.0,9933.0,1.347825e+05,coolcatsnft,False,http://coolcatsnft.com,NaN,https://raritysniper.com/cool-cats,assets_cool-cats
2016,1629000000000,0.950439,uncategorized,ethereum,forgotten-runes-wizards-cult,0x521f9c7505005cfa19a8e5786a9c3c9c9f5e6f42,https://discord.gg/F7WbxwJuZC,1.05590,True,1,...,330.513760,189.0,9518.0,1.597023e+04,forgottenrunes,False,http://forgottenrunes.com,NaN,https://raritysniper.com/forgotten-runes-wizar...,assets_forgotten-runes-wizards-cult
2017,1627852965383,49.282925,uncategorized,ethereum,cryptopunks,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,https://discord.gg/tQp4pSE,NaN,True,768,...,0.000000,0.0,9999.0,1.105317e+06,larvalabs,False,https://www.larvalabs.com/cryptopunks,NaN,https://raritysniper.com/cryptopunks,assets_cryptopunks
2018,1627841943556,11.000597,uncategorized,ethereum,veefriends,0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb,NaN,5.09990,True,763,...,1431.809110,273.0,10255.0,6.138333e+04,veefriends,False,https://veefriends.com/,veefriends,https://raritysniper.com/veefriends,assets_veefriends


In [ ]:
# define fuction get_all_nfts
# iterate while function where hits > 0 
# create variable nft_keys, where there are only required keys [collectionId, CollectionName, nftId, rarityScore, rank]
# payload contains a search query for a specific collection of items.

def get_all_nfts(idx, name):
    collection_nfts = []
    page = 1
    payload = {
        "searches": [
            {
                "sort_by": "rank:asc,nftId:asc",
                "collection": name,
                "q": "*",
                "max_facet_values": 83,
                "page": page,
                "per_page": 250
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    r = requests.post(api_url, json=payload, headers=headers)
    while len(r.json()['results'][0]['hits']) > 0:
        nfts = r.json()['results'][0]['hits']
        for nft in nfts:
            nft_key = nft['document']['collectionId'], nft['document']['collectionName'], nft['document']['nftId'], nft['document']['rarityScore'] ,nft['document']['rank']
            collection_nfts.append(list(nft_key)) # extract values from dictionaries
        print(f'processed: {idx}, collection_name: {name}, page: {page}')
        page += 1
        payload["searches"][0]["page"] = page
        r = requests.post(api_url, json=payload, headers=headers)
    return collection_nfts


# define fucntion dum_csv
# create file csv file [collection_assets_batch_1.csv]
# take in a list of rows (nfts) and writes them to a CSV file named "collection_assets_batch_1.csv".

def dump_csv(nfts):
    file_exists = os.path.isfile('collection_assets_batch_1.csv')
    mode = 'a' if file_exists else 'w'
    with open('collection_assets_batch_1.csv', mode, newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists: # write headers if file doesn't exist
            writer.writerow(['collectionId', 'collectionName', 'nftId', 'rarityScore', 'rank'])
        for row in nfts:
            writer.writerow(row)

# code loops through the first 500 items of the 'assetsname' column in a DataFrame named 'collection_df'.
# create a empty list named [all_nft]
# returned nfts are appended to a list called all_nfts.

all_nfts=[]
for idx, asset_name in enumerate(collection_df['assetsname']):
    nfts = get_all_nfts(idx, asset_name)
    for nft in nfts:
        all_nfts.append(nft)
    if (idx + 1) % 10 == 0: # After every 10th iteration the dump_csv function is called.
        dump_csv(all_nfts) # Passing all_nfts as the argument
        all_nfts=[] # all_nfts is reset to an empty list
dump_csv(all_nfts)